# 2. Biocellion Sensitivity Analysis - Run
This notebook defines functions to run simulations for the Biocellion ABM models by the CMMC that can be found in github.com/TheCMMC. For more information on the CMMC, visit www.thecmmc.org.  


In [1]:
import os
import re

import pandas as pd
import xml.etree.ElementTree as ET

from pathlib import Path
from datetime import datetime
from itertools import chain

## User settings

In [2]:
# Set this data folder to the place where you cloned the repository https://github.com/TheCMMC/biocellion-data
MODEL_FOLDER = Path("/home/jaroknor/NLeSC/InSilicoMeat/Biocellion/Biocellion-3.1/biocellion-3.1/biocellion-user/ABM-microcarriers")
TOOLS_FOLDER = Path("/home/jaroknor/NLeSC/InSilicoMeat/Biocellion/biocellion-tools")

## Run biocellion simulations
Now that we have sampled the parameters, we can run all simulations for these parameter sets. We need to fill in the paramater values in the correct place in the biocellion files and compile the model. Then run the model and store the results. The output of each run will get a unique time stamp and a parameter file that contains the parameters for that run. 

In [29]:
class Model:
    def __init__(self, param_vector, model_folder, output_base_folder, problem, param_info, num_steps=None):
        self.param_vector = param_vector
        self.model_folder = model_folder
        self.output_base_folder = output_base_folder
        self.problem = problem
        self.param_info = param_info
        self.num_steps = num_steps
        
        self.init_time = None
        self.timestamp = None
        self.xml = None
        self.xml_path = None
        self.output_location = None
        
        self._set_timestamp()
        
        
    def _set_timestamp(self):
        self.init_time = datetime.now()
        self.timestamp = self.init_time.strftime("%Y%m%d-%H%M%S")


    def _prepare_xml(self):
        """Read original xml file and write to timestamped new file"""

        base_xml_path = self.model_folder / "run_model.xml"
        self.xml = ET.parse(base_xml_path)
        self.xml_path = base_xml_path.with_stem(f"run_model_{self.timestamp}")
        self.xml.write(self.xml_path)

    def _prepare_output_location(self):
        """Creates new timestamped folder and renames output paths in xml file"""

        self.output_location = self.output_base_folder / f"output_{self.timestamp}"
        os.mkdir(self.output_location)

        #  rename the output paths 
        root = self.xml.getroot()
        for element in chain(root.iter("output"), root.iter("stdout")):
            element.set("path", str(self.output_location))
        self.xml.write(self.xml_path)
        
    def _write_simulation_info(self):
        param_sample_df = pd.DataFrame([self.param_vector], columns=self.problem["names"])
        param_sample_df.to_csv(self.output_location / f"parameters_{self.timestamp}.csv")

    def _set_num_steps(self):
        """Creates new timestamped folder and renames output paths in xml file"""

        root = self.xml.getroot()
        for element in root.iter("time_step"):
            element.set("end", str(self.num_steps))
        self.xml.write(self.xml_path)        
        

    def _enter_parameter(self, index, value):
        info = self.param_info.loc[index]
        parameter_loc = self.model_folder / info["file"]
        param_type, param_name = info["type"], info["name"]
        string = f"const {param_type} {param_name} = .+ ;" 
        pattern = re.compile(string)
        new_string = f"const {param_type} {param_name} = {value} ;"
        
        with open(parameter_loc, "r") as file:
            content = file.read()
            assert re.search(pattern, content)
            content = re.sub(pattern, new_string, content)
            
        with open(parameter_loc, "w") as file:
            file.write(content) 
        
        
    def prepare(self):
        """Enters the parameter values and output locations in the right file and place, then compiles the model."""

        print(f"Preparing model at {self.init_time}")
        self._prepare_xml()
        self._prepare_output_location()
        self._write_simulation_info()
        if self.num_steps:
            self._set_num_steps()
        for i, value in enumerate(self.param_vector):
            self._enter_parameter(i, value)
        print(f"Output will be collected in: \n {self.output_location}")


    def build(self):
        """Builds the model using make, after cleaning up the previous build"""
        print("Building the model...")
        cwd = os.getcwd()
        os.chdir(self.model_folder)
        
        os.system("make clean")
        os.system("make -s")

        os.chdir(cwd)

        
    def run(self):
        print(f"Starting simulation with {self.xml_path.stem}")
        cwd = os.getcwd()
        os.chdir(self.model_folder)
        !biocellion {self.xml_path}
        os.chdir(cwd)
        os.rename(self.xml_path, self.output_location / self.xml_path.name)


In [30]:
output_base_folder = MODEL_FOLDER / "output"
model = Model(param_sample[0], MODEL_FOLDER, output_base_folder, problem, param_info, num_steps=101)
model.prepare()
model.build()
model.run()

Preparing model at 2021-06-21 17:33:00.371283
Output will be collected in: 
 /home/jaroknor/NLeSC/InSilicoMeat/Biocellion/Biocellion-3.1/biocellion-3.1/biocellion-user/ABM-microcarriers/output/output_20210621-173300
Building the model...
Starting simulation with run_model_20210621-173300
WARNING, this binary does not support distributed memory parallelism, and "num_node_groups" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "num_nodes_per_group" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "num_sockets_per_node" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "load_balance" (attribute of "interval") will be ignored.
Reading Velocity_220rpm.txt...  read 291016 points.


In [65]:
def run_batch(param_vectors, problem, param_info, model_folder, num_steps=101):
    # Create the parameter sample file to be able to rerun the same batch
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    batch_output_folder = model_folder / f"output_SA_{timestamp}"
    os.mkdir(batch_output_folder)
    param_sample_df = pd.DataFrame(param_vectors, columns=problem["names"])
    param_sample_df.to_csv(batch_output_folder / f"parameter_sample_{timestamp}.csv")
    param_info.to_csv(batch_output_folder / f"parameter_info_{timestamp}.csv")
    
    for i, param_vector in enumerate(param_vectors):
        print(f"Starting run for parameter set {i}")
        model = Model(param_vector, model_folder, batch_output_folder, problem, param_info, num_steps=num_steps)
        model.prepare()
        model.build()
        model.run()
        print("---------------- \n")

In [ ]:
run_batch(param_sample, problem, param_info, MODEL_FOLDER, num_steps=120000)

Starting run for parameter set 0
Preparing model at 2021-06-04 17:59:35.226399
Output will be collected in: 
 /home/jaroknor/NLeSC/InSilicoMeat/Biocellion/Biocellion-3.1/biocellion-3.1/biocellion-user/ABM-microcarriers/output_20210604-175935/output_20210604-175935
Building the model...
Starting simulation with run_model_20210604-175935
WARNING, this binary does not support distributed memory parallelism, and "num_node_groups" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "num_nodes_per_group" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "num_sockets_per_node" (child element of "system") will be ignored.
WARNING, this binary does not support distributed memory parallelism, and "load_balance" (attribute of "interval") will be ignored.
Reading Velocity_220rpm.txt...  read 291016 points.
termination took 0.0002 seconds.
Simulation finished.
----